In [1]:
import torch
from avalanche.training.supervised import LwF
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import TensorboardLogger
from torch.utils.tensorboard import SummaryWriter
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, timing_metrics, forgetting_metrics,class_accuracy_metrics
from avalanche.benchmarks.classic import CORe50
from avalanche.logging import InteractiveLogger
from SimpleCNN import *
from avalanche.benchmarks.utils import as_avalanche_dataset

c:\Users\tellg\Documents\Universita\VISIOPE\2023-2024\Progetto\ContinualLearning\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torchvision import transforms

# Trasformazione per ridurre la risoluzione delle immagini
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Riduci la risoluzione a 64x64 (o altra risoluzione desiderata)
    transforms.ToTensor()
])

In [3]:
# Caricamento del dataset CORe50
benchmark = CORe50(scenario="nc",run=0,object_lvl=False,train_transform=transform,eval_transform=transform)
log_dir = './tb_data/experiment_SimpleCNN_LwF'

# Creazione del logger di TensorBoard per run1
tb_logger_run1 = TensorboardLogger(tb_log_dir=log_dir)

# Lista dei modelli
print("Classi Presenti : "+str(benchmark.n_classes))
model = SimpleCNN(num_classes=benchmark.n_classes)

print(f"Training model: {model.__class__.__name__}")


Loading labels...
Loading LUP...
Loading labels names...
Files already downloaded and verified
Classi Presenti : 10
Training model: SimpleCNN


In [4]:

# Plugin per la valutazione
eval_plugin = EvaluationPlugin(    
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),\
    class_accuracy_metrics(stream=True),\
    loggers=[tb_logger_run1])

# Imposta l'ottimizzatore
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Definisci la strategia LwF
lwf_strategy = LwF(
    model,
    optimizer,
    criterion,
    alpha=0.5,  # Peso della loss LwF, modifica secondo necessità
    temperature=2.0,  # Temperatura per la distillazione, modifica secondo necessità
    train_mb_size=100,  # Dimensione del mini-batch
    train_epochs=1,  # Numero di epoche di addestramento
    eval_mb_size=32,  # Dimensione del mini-batch per la valutazione
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    evaluator=eval_plugin
)

for experience in benchmark.train_stream:
    print(f"Training on experience {experience.current_experience}")
    lwf_strategy.train(experience)
    print("Training completed")

    print("Evaluating on test stream")
    lwf_strategy.eval(benchmark.test_stream)
    print("Evaluation completed")


c:\Users\tellg\Documents\Universita\VISIOPE\2023-2024\Progetto\ContinualLearning\.venv\lib\site-packages\avalanche\training\templates\base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the LwF.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


Training on experience 0
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 1
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 2
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 3
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 4
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 5
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 6
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 7
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 8
Training completed
Evaluating on test stream
Evaluation completed
